# Get Planck MCMC parameter Covariance Matrices and Process into Fisher matrices

In [44]:
# load the data
planckDir = 'Planck_data/2013/'
#covmatFile = 'base_planck_lowl_post_lensing.covmat'
#cormatFile = 'base_planck_lowl_post_lensing.corr'
#paramNamesFile = 'base_planck_lowl_post_lensing.paramnames'

covmatFile = 'base_planck_lowl_lowLike_highL_post_lensing.covmat'
cormatFile = 'base_planck_lowl_lowLike_highL_post_lensing.corr'
paramNamesFile = 'base_planck_lowl_lowLike_highL_post_lensing.paramnames'

# data from page:
# https://wiki.cosmos.esa.int/planckpla/index.php/Cosmological_Parameters

covmat = np.loadtxt(planckDir+covmatFile)
cormat = np.loadtxt(planckDir+cormatFile)
#paramNames = np.loadtxt(planckDir+paramNamesFile)

text_file = open(planckDir+paramNamesFile, "r")
lines = text_file.readlines()
print lines
print len(lines)
text_file.close()

for lineNum in range(len(lines)):
    print 'line ',lineNum,': ',lines[lineNum]

[' omegabh2\t\\Omega_b h^2\n', ' omegach2\t\\Omega_c h^2\n', ' theta\t100\\theta_{MC}\n', ' tau\t\\tau\n', ' ns\tn_s\n', ' logA\t{\\rm{ln}}(10^{10} A_s)\n', ' aps100\tA^{PS}_{100}\n', ' aps143\tA^{PS}_{143}\n', ' aps217\tA^{PS}_{217}\n', ' acib143\tA^{CIB}_{143}\n', ' acib217\tA^{CIB}_{217}\n', ' asz143\tA^{tSZ}_{143}\n', ' psr\tr^{PS}_{143\\times217}\n', ' cibr\tr^{CIB}_{143\\times217}\n', ' ncib\t\\gamma^{CIB}\n', ' cal0\tc_{100}\n', ' cal2\tc_{217}\n', ' xi\t\\xi^{tSZ-CIB}\n', ' aksz\tA^{kSZ}\n', ' bm_1_1\t\\beta^1_1\n', ' a_ps_act_148\tA^{PS,ACT}_{148}\n', ' a_ps_act_217\tA^{PS,ACT}_{218}\n', ' a_ps_spt_95\tA^{PS,SPT}_{95}\n', ' a_ps_spt_150\tA^{PS,SPT}_{150}\n', ' a_ps_spt_220\tA^{PS,SPT}_{220}\n', ' r_ps_spt_95x150\tr^{PS}_{95\\times150}\n', ' r_ps_spt_95x220\tr^{PS}_{95\\times220}\n', ' r_ps_150x220\tr^{PS}_{150\\times220}\n', ' act_dust_s\tA^{ACTs}_{\\rm dust}\n', ' act_dust_e\tA^{ACTe}_{\\rm dust}\n', ' cal_acts_148\ty_{ACTs}^{148}\n', ' cal_acts_217\ty_{ACTs}^{217}\n', ' cal_

In [45]:
# get indices of interest from the above output
print 'covmat shape: ', covmat.shape
print 'cormat shape: ', cormat.shape

# covariance matrix is only 20x20.  If I use this then I'll have to use theta_mc instead of H0.
selectIndices = [0,1,2,5,4,3] #covmat
print 'covmat: ',[lines[si] for si in selectIndices]

# correlation matrix is 46x46.  Too bad this isn't also a covariance matrix.
#selectIndices = [0,1,25,27,4,3]  #cormat
#print 'cormat: ',[lines[si] for si in selectIndices]


covmat shape:  (37, 37)
cormat shape:  (63, 63)
covmat:  [' omegabh2\t\\Omega_b h^2\n', ' omegach2\t\\Omega_c h^2\n', ' theta\t100\\theta_{MC}\n', ' logA\t{\\rm{ln}}(10^{10} A_s)\n', ' ns\tn_s\n', ' tau\t\\tau\n']


In [46]:
# can I get a jacobian to translate theta_mc to H0?
# yes.  Use discrete derivative with CAMB.  
# dtheta/dparams vector below from notebook "Rebuild covar with noise"

In [57]:
#dtheta/dparams
nCosParams = 6
paramList = ['ombh2','omch2','H0','As','ns','tau']
dthetas = [-0.024833399132992862, 0.010457065703734872, 2.9621955038288544e-05, 0.0, 0.0, 0.0]

# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(nCosParams))
jacobian[mcIndex,:] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian

[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.        ]
 [-2.48333991  1.04570657  0.0029622   0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]


In [ ]:
# jacobian-it-up
#invCovPlanck = np.dot(jacobian.T,np.dot(invCovPlanck,jacobian))
#print invCovPlanck

In [47]:
# the best-fit parameter values for planck-lowl-post-lensing
# from https://wiki.cosmos.esa.int/planckpla/images/6/62/Comparetables_limit68.pdf
bestFit = [0.02228,0.1174,68.53,3.042,0.9674,0.0682]  # cormat version.  look at table again to get covmat version
# but, the 3 index entry 3.042 is in ln 10^10 As, not 10^9 As

In [48]:
# fix the nuissance parameters

# invert covariance matrix
invCov = np.linalg.inv(covmat)
print invCov.shape
print np.array([invCov[si,si] for si in selectIndices])

#subInvCov = [invCov[si1][si2] for si1 in selectIndices for si2 in selectIndices]
#subInvCov = [[invCov[:,si1] for si1 in selectIndices[si2]] for si2 in selectIndices]
#subInvCov1 = [invCov[si1] for si1 in selectIndices]
#print subInvCov1
#subInvCov = [subInvCov1[:,si2] for si2 in selectIndices]

nParams = selectIndices.__len__()
subInvCov = np.empty((nParams,nParams))
for si1 in range(6):
    for si2 in range(6):
        subInvCov[si1,si2] = invCov[selectIndices[si1],selectIndices[si2]]
#print subInvCov
print np.diag(subInvCov)

(37, 37)
[ 33650264.75304625   1729869.64336173   4962497.99100187
    395050.69626645    193655.85777067   1559034.31753978]
[ 33650264.75304625   1729869.64336173   4962497.99100187
    395050.69626645    193655.85777067   1559034.31753978]


In [49]:
# re-invert and get sigmas
sigmas = np.sqrt(np.diag(np.linalg.inv(subInvCov)))
print sigmas

[ 0.00021205  0.00210681  0.00059998  0.02239803  0.00528772  0.0123382 ]


In [51]:
# save that sub-Fisher matrix to use in another notebook
# I will want it in "Rebuild covar with noise"
np.savetxt('PlanckInvCovFromMCMC.txt',subInvCov)